ADD MARK TAG FUNCTION:

In [2]:
def add_mark_tag(current_matched_index,searched_pattern,text):
   target_text=""
   skip=False
   for i in range(len(current_matched_index)):
      if not (skip):
         if i == 0:
            k=0
         else:
            k=current_matched_index[i-1]+len(searched_pattern)+13
         if i+1>len(current_matched_index)-1:
            e=len(text)
         else:
            e=current_matched_index[i+1]
            if (current_matched_index[i] + len(searched_pattern)-1==current_matched_index[i+1]):
               if i+2 > len(current_matched_index)-1:
                  e=len(text)
               else:
                  e=current_matched_index[i+2]
               target_text+=text[k:current_matched_index[i]] + "<mark>" + searched_pattern + searched_pattern[1:] + "</mark>" + text[current_matched_index[i]+len(searched_pattern)*2-1:e]
               skip=True
         if not (skip):
            target_text += text[k:current_matched_index[i]] + "<mark>" + searched_pattern + "</mark>" + text[current_matched_index[i]+len(searched_pattern):e]
      else:
         if i+1==len(current_matched_index)-1:
            e=len(text)
            target_text+=text[current_matched_index[i]+len(searched_pattern)*2-1:e]
         skip=False
   return target_text
add_mark_tag([3,5,15,22,24],"101","00010101100000010100001010100000000")  

'000<mark>10101</mark>1000000<mark>101</mark>0000<mark>10101</mark>00000000'

BRUTE FORCE ALGORITHM:

In [3]:
import time    
def BruteForce(input_path, searched_pattern):
    with open(input_path, "r", encoding='utf-8') as f:
        text= f.read()
    #text="abcabcabbbfgergoabckewşrkgeşrgerabcabcw4erwrwerabc"
    #text="10000100001"
    len_searched_pattern = len(searched_pattern)
    len_text = len(text)
    print("Len: "+str(len_text))
    current_index = 0
    current_matched_index=[]
    match_count=0
    number_of_comparisons=0
    elapsed_time=0
    target_text=""
    start=time.time()
    matched=False
    while(current_index < len_text-len_searched_pattern+1):
        if(matched):
            current_matched_index.append(current_index)
            current_index+=1
            #print("Genel Eşleşme başarılı: " + text[(current_index-len_searched_pattern):current_index])
            match_count+=1
        for index_of_pattern in range(len_searched_pattern):
            end_point = current_index+index_of_pattern
            if(end_point>len_text-len_searched_pattern+1):
                break
            number_of_comparisons+=1
            """
            print("K1:" + searched_pattern[index_of_pattern])
            print("K2:" + text[end_point]) """
            if(searched_pattern[index_of_pattern] != text[end_point]):
                current_index=current_index+1
                #print("Eşleşme başarısız")
                matched=False
                break
            else:
                #print("Eşleşme başarılı")
                matched=True
    print("Number of comparison: " + str(number_of_comparisons))
    print("Match count: " + str(match_count))
    elapsed_time=time.time()-start
    print("Elapsed time: " + str(elapsed_time))
    print(current_index)
    target_text=add_mark_tag(current_matched_index,searched_pattern,text)
    with open(input_path + "_output.html",'w') as file:
        file.write(target_text)


In [4]:
input_path ="test_bitstring_w.html"
BruteForce(input_path,"100101")

Len: 1048602
Number of comparison: 2120888
Match count: 4301
Elapsed time: 0.5306084156036377
1048597


CREATING BAD SYMBOL TABLE FOR BAYER-MOORE AND HORSPOOL

In [5]:
def get_bad_symbol_table(searched_pattern):
    searched_pattern = searched_pattern[::-1]
    bad_symbol_dict={}
    for val in range(1,len(searched_pattern)):
        if(searched_pattern[val] not in bad_symbol_dict.keys()):
            bad_symbol_dict[searched_pattern[val]] = val
    bad_symbol_dict['Other letters'] = len(searched_pattern)
    return bad_symbol_dict

HORSPOOL ALGORITHM:

In [6]:
def HorsPool(input_path, searched_pattern):
    with open(input_path, "r", encoding='utf-8') as f:
        text= f.read()
    #text="10000100001"
    len_searched_pattern = len(searched_pattern)
    len_text = len(text)
    current_matched_index=[]
    current_index = 0
    number_of_comparisons=0
    elapsed_time=0
    match_count=0
    target_text=""
    start=time.time()
    matched=False
    shift_dict = get_bad_symbol_table(searched_pattern)
    print(shift_dict)
    while(current_index < len_text-len_searched_pattern+1):
        if(matched):
            current_matched_index.append(current_index)
            current_index+=1
            #print("Genel Eşleşme başarılı: " + text[(current_index-len_searched_pattern):current_index])
            match_count+=1
        for index_of_pattern in range(len_searched_pattern-1,-1,-1):
            end_point = current_index+index_of_pattern
            last_char_of_comparison = current_index+len_searched_pattern-1
            if(end_point>len_text-len_searched_pattern+1):
                current_index=len_text-1
                break
            number_of_comparisons+=1
            """
            print("K1:" + searched_pattern[index_of_pattern])
            print("K2:" + text[end_point]) """
            if(searched_pattern[index_of_pattern] != text[end_point]):
                shift_count=0
                if(text[last_char_of_comparison] in shift_dict.keys()):
                    shift_count=shift_dict[text[last_char_of_comparison]]
                else:
                    shift_count=shift_dict['Other letters']
                #print("KARŞILAŞTIRMA (SON KARAKTER): " + text[last_char_of_comparison])
                current_index=current_index+shift_count
                #print("Eşleşme başarısız")
                matched=False
                break
            else:
                #print("Eşleşme başarılı")
                matched=True
    print("Number of comparison: " + str(number_of_comparisons))
    print("Match count: " + str(match_count))
    elapsed_time=time.time()-start
    print("Elapsed time: " + str(elapsed_time))
    target_text=add_mark_tag(current_matched_index,searched_pattern,text)
    with open(input_path + "_output.html",'w') as file:
        file.write(target_text)


BUILDING GOOD SUFFIX TABLE AND DETECTING PREFIX-SUFFIX OCCURENCES FOR BAYER-MOORE:

In [7]:
def is_longest_occurence(good_suffix_table,k,matched_pattern,searched_pattern):
     if(k==2):
         return False
     if((len(good_suffix_table.keys())>1) and (good_suffix_table[k-1][1] in matched_pattern)) and (good_suffix_table[k-1][1] in searched_pattern[0:len(good_suffix_table[k-1][1])]):
         return True
     else:
         return is_longest_occurence(good_suffix_table,k-1,matched_pattern,searched_pattern)
def build_good_suffix_table(searched_pattern):
    len_searched_pattern = len(searched_pattern)
    good_suffix_table = {}
    k=1
    while(k<=len_searched_pattern):
        matched_pattern = searched_pattern[(len_searched_pattern-k):len(searched_pattern)]
        len_matched_pattern = len(matched_pattern)
        if(k-1>=0) and len_searched_pattern-k>=1:
            preciding_char = searched_pattern[len_searched_pattern-k-1]
        else:
            preciding_char=" "
        current_char=""
        """
        print("Matched_pattern: " + matched_pattern)
        print("Preceeding char: " +preciding_char)
        """
        current_index=0
        reference_index=-1
        while(current_index<len_searched_pattern-1):
            for i in range(current_index,len_searched_pattern+1,k):
                if (searched_pattern[i:i+k] == matched_pattern):
                    """
                    if matched_pattern=="00101":
                        print(matched_pattern + " founded in: " +str(i) + "-" + str(i+k))
                    """
                    if(i == 0):
                        reference_index=0
                    elif(preciding_char != searched_pattern[i-1]):
                        reference_index=i
            current_index+=1
        #print("Reference_index:" + str(reference_index))
        if(reference_index==-1 or k==len_searched_pattern) :
            #print(searched_pattern[0:len(good_suffix_table[k-1][1])])
            if((len(good_suffix_table.keys())>1) and (good_suffix_table[k-1][1] in matched_pattern)) and is_longest_occurence(good_suffix_table,k,matched_pattern,searched_pattern):
                #if(matched_pattern=="00101"):
                    #print("ferfewr")
                d = good_suffix_table[k-1][0]
            elif(matched_pattern[len_matched_pattern-1] == searched_pattern[0]): #Last char-first char match
                #print("dwddddddd")
                d=len_searched_pattern-1
            else:
                d=len_searched_pattern
        else:
            #print("aaaaa" + matched_pattern)
            d=len_searched_pattern-reference_index-1-(len_matched_pattern-1)
        lst=[d,matched_pattern]
        good_suffix_table[k]=lst
        """
        print("d:" + str(d))
        print("k:"+str(k))
        print("--------\n")
        """
        k+=1
    return good_suffix_table
build_good_suffix_table("101010")

{1: [6, '0'],
 2: [4, '10'],
 3: [4, '010'],
 4: [2, '1010'],
 5: [2, '01010'],
 6: [2, '101010']}

BAYER-MOORE ALGORITHM:

In [8]:
def Bayer_Moore(input_path,searched_pattern):
    with open(input_path, "r", encoding='utf-8') as f:
        text= f.read()
    #text="10000100001"
    len_searched_pattern = len(searched_pattern)
    len_text = len(text)
    current_matched_index=[]
    current_index = 0
    number_of_comparisons=0
    elapsed_time=0
    k=0
    match_count=0
    target_text=""
    start=time.time()
    matched=False
    shift_dict = get_bad_symbol_table(searched_pattern)
    shift_dict_good = build_good_suffix_table(searched_pattern)
    print(shift_dict_good)
    print(shift_dict)
    while(current_index < len_text-len_searched_pattern+1):
        #print(current_index)
        if(matched):
            current_matched_index.append(current_index)
            current_index+=1
            #print("Genel Eşleşme başarılı: " + text[(current_index-len_searched_pattern):current_index])
            match_count+=1
        k=0
        for index_of_pattern in range(len_searched_pattern-1,-1,-1):
            end_point = current_index+index_of_pattern
            if(end_point>len_text-len_searched_pattern+1):
                current_index=len_text-1
                break
            number_of_comparisons+=1
            """
            print("K1:" + searched_pattern[index_of_pattern])
            print("K2:" + text[end_point]) """
            if(searched_pattern[index_of_pattern] != text[end_point]):
                shift_count=0
                if(text[end_point] in shift_dict.keys()):
                    d1=shift_dict[text[end_point]]
                else:
                    d1=shift_dict['Other letters']
                if(k>0):
                    d2=shift_dict_good[k][0]
                else:
                    d2=0
                d1 = max(d1-k,1)
                shift_count=max(d1,d2)
                #print("KARŞILAŞTIRMA (SON KARAKTER): " + text[last_char_of_comparison])
                current_index=current_index+shift_count
                #print("Eşleşme başarısız")
                matched=False
                k=0
                break
            else:
                #print("Eşleşme başarılı")
                matched=True
                k+=1
    print("Number of comparison: " + str(number_of_comparisons))
    print("Match count: " + str(match_count))
    elapsed_time=time.time()-start
    print("Elapsed time: " + str(elapsed_time))
    target_text=add_mark_tag(current_matched_index,searched_pattern,text)
    with open(input_path + "_output.html",'w') as file:
        file.write(target_text)

TESTING IMPLEMENTED ALGORITHMS:

In [9]:
input_path ="test_bitstring_w.html"
BruteForce(input_path,"100101")

Len: 1048602
Number of comparison: 2120888
Match count: 4301
Elapsed time: 0.5685076713562012
1048597


In [10]:
HorsPool(input_path,"100101")

{'0': 1, '1': 2, 'Other letters': 6}
Number of comparison: 1387716
Match count: 4301
Elapsed time: 0.4787464141845703


In [11]:
Bayer_Moore(input_path,"100101")

{1: [5, '1'], 2: [2, '01'], 3: [5, '101'], 4: [5, '0101'], 5: [5, '00101'], 6: [5, '100101']}
{'0': 1, '1': 2, 'Other letters': 6}
Number of comparison: 635964
Match count: 4301
Elapsed time: 0.2682816982269287


UniformBit-1MB-101010

In [30]:
input_path="Tests/UniformBit-1MB-101010/test_bitstring.html"
BruteForce(input_path,"101010")

Len: 1048602
Number of comparison: 2065748
Match count: 16483
Elapsed time: 0.5754640102386475
1048597


In [31]:
HorsPool(input_path,"101010")

{'1': 1, '0': 2, 'Other letters': 6}
Number of comparison: 1558194
Match count: 16483
Elapsed time: 0.5834665298461914


In [32]:
Bayer_Moore(input_path,"101010")

{1: [6, '0'], 2: [4, '10'], 3: [4, '010'], 4: [2, '1010'], 5: [2, '01010'], 6: [2, '101010']}
{'1': 1, '0': 2, 'Other letters': 6}
Number of comparison: 859410
Match count: 16483
Elapsed time: 0.3979635238647461


UniformBit-1MB-101010101010

In [15]:
input_path="Tests/UniformBit-1MB-101010101010/test_bitstring.html"
BruteForce(input_path,"101010101010")

Len: 1048602
Number of comparison: 2096991
Match count: 230
Elapsed time: 0.5626697540283203
1048591


In [16]:
HorsPool(input_path,"101010101010")

{'1': 1, '0': 2, 'Other letters': 12}
Number of comparison: 1573429
Match count: 230
Elapsed time: 0.6113910675048828


In [17]:
Bayer_Moore(input_path,"101010101010")

{1: [12, '0'], 2: [10, '10'], 3: [10, '010'], 4: [8, '1010'], 5: [8, '01010'], 6: [6, '101010'], 7: [6, '0101010'], 8: [4, '10101010'], 9: [4, '010101010'], 10: [2, '1010101010'], 11: [2, '01010101010'], 12: [2, '101010101010']}
{'1': 1, '0': 2, 'Other letters': 12}
Number of comparison: 399418
Match count: 230
Elapsed time: 0.21644830703735352


UnifromBit-2MB-101010

In [18]:
input_path="Tests/UnifromBit-2MB-101010/test_bitstring.html"
BruteForce(input_path,"101010")

Len: 2097178
Number of comparison: 4129498
Match count: 32700
Elapsed time: 1.17787504196167
2097173


In [19]:
HorsPool(input_path,"101010")

{'1': 1, '0': 2, 'Other letters': 6}
Number of comparison: 3112631
Match count: 32700
Elapsed time: 1.2117838859558105


In [20]:
Bayer_Moore(input_path,"101010")

{1: [6, '0'], 2: [4, '10'], 3: [4, '010'], 4: [2, '1010'], 5: [2, '01010'], 6: [2, '101010']}
{'1': 1, '0': 2, 'Other letters': 6}
Number of comparison: 1717205
Match count: 32700
Elapsed time: 1.0102980136871338


UnifromBit-4MB-101010

In [21]:
input_path="Tests/UnifromBit-4MB-101010/test_bitstring.html"
BruteForce(input_path,"101010")

Len: 4194330
Number of comparison: 8254745
Match count: 65297
Elapsed time: 2.3098528385162354
4194325


In [22]:
HorsPool(input_path,"101010")

{'1': 1, '0': 2, 'Other letters': 6}
Number of comparison: 6225127
Match count: 65297
Elapsed time: 2.2759270668029785


In [23]:
Bayer_Moore(input_path,"101010")

{1: [6, '0'], 2: [4, '10'], 3: [4, '010'], 4: [2, '1010'], 5: [2, '01010'], 6: [2, '101010']}
{'1': 1, '0': 2, 'Other letters': 6}
Number of comparison: 3432520
Match count: 65297
Elapsed time: 1.7612874507904053


HighProbabilityOf0inInput-1MB-100010

In [36]:
input_path="Tests/HighProbabilityOf0inInput-1MB-100010/test_bitstring.html"
BruteForce(input_path,"100010")

Len: 1048602
Number of comparison: 1688324
Match count: 17112
Elapsed time: 0.47373056411743164
1048597


In [37]:
HorsPool(input_path,"100010")

{'1': 1, '0': 2, 'Other letters': 6}
Number of comparison: 1560754
Match count: 17112
Elapsed time: 0.5086374282836914


In [38]:
Bayer_Moore(input_path,"100010")

{1: [2, '0'], 2: [4, '10'], 3: [4, '010'], 4: [4, '0010'], 5: [4, '00010'], 6: [4, '100010']}
{'1': 1, '0': 2, 'Other letters': 6}
Number of comparison: 1248875
Match count: 17112
Elapsed time: 0.5465366840362549


HighProbabilityOf1inInput-1MB-100010

In [33]:
input_path="Tests/HighProbabilityOf1inInput-1MB-100010/test_bitstring.html"
BruteForce(input_path,"100010")

Len: 1048602
Number of comparison: 2100730
Match count: 1088
Elapsed time: 0.5824687480926514
1048597


In [34]:
HorsPool(input_path,"100010")

{'1': 1, '0': 2, 'Other letters': 6}
Number of comparison: 1259329
Match count: 1088
Elapsed time: 0.6482658386230469


In [35]:
Bayer_Moore(input_path,"100010")

{1: [2, '0'], 2: [4, '10'], 3: [4, '010'], 4: [4, '0010'], 5: [4, '00010'], 6: [4, '100010']}
{'1': 1, '0': 2, 'Other letters': 6}
Number of comparison: 937949
Match count: 1088
Elapsed time: 0.6073729991912842
